# Import CSV data and Overview

In [1]:
import pandas as pd

# CSV file URL
url = 'https://raw.githubusercontent.com/Scherwiin/AI_Darsman/main/Datasets/loans.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(url)

# Display the first few rows of the DataFrame
print(df.head())

# Display information about the DataFrame
print(df.info())

   client_id loan_type  loan_amount  repaid  loan_id  loan_start    loan_end  \
0      46109      home        13672       0    10243  2002-04-16  2003-12-20   
1      46109    credit         9794       0    10984  2003-10-21  2005-07-17   
2      46109      home        12734       1    10990  2006-02-01  2007-07-05   
3      46109      cash        12518       1    10596  2010-12-08  2013-05-05   
4      46109    credit        14049       1    11415  2010-07-07  2012-05-21   

   rate  
0  2.15  
1  1.25  
2  0.68  
3  1.24  
4  3.13  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 443 entries, 0 to 442
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   client_id    443 non-null    int64  
 1   loan_type    443 non-null    object 
 2   loan_amount  443 non-null    int64  
 3   repaid       443 non-null    int64  
 4   loan_id      443 non-null    int64  
 5   loan_start   443 non-null    object 
 6   loan_end     4

# Detection and processing of missing values

In [2]:
# Detect missing values
missing_values = df.isna()

def apply_methods_to_columns(df, methods):
  """
  Apply methods to corrresponding columns in the DataFrame.

  Args:
  - df (DataFrame): The DataFrame to process.
  -methods (dict): A dictionary where the keys are column names and values are methods t oapply to those columns.

  Returns:
  - DataFrame: The DataFrame with methods applied to corresponding columns.
  """
  for column, method in methods.items():
    df[column] = method(df[column])
  return df

# Change loan_start and loan_end column data type to datetime
df['loan_start'] = pd.to_datetime(df['loan_start'])
df['loan_end'] = pd.to_datetime(df['loan_end'])

# Define methods for each column
methods = {
    'client_id': lambda x: x.fillna(method='ffill'),
    'loan_type': lambda x: x.fillna(method='bfill'),
    'loan_amount': lambda x: x.fillna(x.mean()),
    'repaid': lambda x: x.fillna(x.mode()),
    'loan_id': lambda x: x.fillna(x.max()+1),
    'loan_start': lambda x: x.dropna(axis=0),
    'loan_end': lambda x: x.dropna(axis=0),
    'rate': lambda x: x.fillna(x.mode())
}

# Apply methods to corresponding columns
df_processed = apply_methods_to_columns(df, methods)

# Check the result
print(df_processed)
print(df.info())

     client_id loan_type  loan_amount  repaid  loan_id loan_start   loan_end  \
0        46109      home        13672       0    10243 2002-04-16 2003-12-20   
1        46109    credit         9794       0    10984 2003-10-21 2005-07-17   
2        46109      home        12734       1    10990 2006-02-01 2007-07-05   
3        46109      cash        12518       1    10596 2010-12-08 2013-05-05   
4        46109    credit        14049       1    11415 2010-07-07 2012-05-21   
..         ...       ...          ...     ...      ...        ...        ...   
438      26945     other        12963       0    10330 2001-11-26 2004-06-11   
439      26945    credit         1728       1    10248 2004-01-27 2005-06-21   
440      26945     other         9329       0    10154 2001-12-17 2004-07-22   
441      26945      home         4197       0    10333 2003-10-16 2005-07-10   
442      26945      home         3643       0    11434 2010-03-24 2011-12-22   

     rate  
0    2.15  
1    1.25  
2  

# Detection and processing of outliers

In [3]:
def delete_outliers(df, column):
  """
  Detect outliers in a DataFrame column using the interquartile range (IQR) method and delete them.

  Args:
  - df (DataFrame): The DataFrame containing the column.
  - column (str): The name of the column to detect outliers in and delete from.

  Returns:
  - DataFrame: The DataFrame with outliers deleted from the specified column.
  """
  # Calculate quartiles
  Q1 = df[column].quantile(0.25)
  Q3 = df[column].quantile(0.75)

  # Calculate interquartile range (IQR)
  IQR = Q3- Q1

  # Define lower and upper bounds
  lower_bound = Q1 - 1.5 * IQR
  upper_bound = Q3 + 1.5 * IQR

  # Filter out rows containing outliers
  df_filtered = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

  return df_filtered

df = df_processed

# Delete outliers from loan_amount and rate column
df_filtered = delete_outliers(df, 'rate')
df_filtered = delete_outliers(df_filtered, 'loan_amount')
df_reindexed = df_filtered.reset_index(drop=True)
# Check the result
print(df_reindexed)
print(df_reindexed.info())

     client_id loan_type  loan_amount  repaid  loan_id loan_start   loan_end  \
0        46109      home        13672       0    10243 2002-04-16 2003-12-20   
1        46109    credit         9794       0    10984 2003-10-21 2005-07-17   
2        46109      home        12734       1    10990 2006-02-01 2007-07-05   
3        46109      cash        12518       1    10596 2010-12-08 2013-05-05   
4        46109    credit        14049       1    11415 2010-07-07 2012-05-21   
..         ...       ...          ...     ...      ...        ...        ...   
435      26945     other        12963       0    10330 2001-11-26 2004-06-11   
436      26945    credit         1728       1    10248 2004-01-27 2005-06-21   
437      26945     other         9329       0    10154 2001-12-17 2004-07-22   
438      26945      home         4197       0    10333 2003-10-16 2005-07-10   
439      26945      home         3643       0    11434 2010-03-24 2011-12-22   

     rate  
0    2.15  
1    1.25  
2  

#Encoding categorical variables

In [4]:
from sklearn.preprocessing import OneHotEncoder

df = df_reindexed

# OneHot encoding for loan_type column
onehot_encoder = OneHotEncoder(sparse_output=False)
onehot_encoded = onehot_encoder.fit_transform(df[['loan_type']])
onehot_encoded_df = pd.DataFrame(onehot_encoded, columns=onehot_encoder.get_feature_names_out(['loan_type']))

# Combine encoded column with the original DataFrame
df_encoded = pd.concat([df.drop(columns=['loan_type']), onehot_encoded_df], axis=1)

# Show the result
print(df_encoded)

     client_id  loan_amount  repaid  loan_id loan_start   loan_end  rate  \
0        46109        13672       0    10243 2002-04-16 2003-12-20  2.15   
1        46109         9794       0    10984 2003-10-21 2005-07-17  1.25   
2        46109        12734       1    10990 2006-02-01 2007-07-05  0.68   
3        46109        12518       1    10596 2010-12-08 2013-05-05  1.24   
4        46109        14049       1    11415 2010-07-07 2012-05-21  3.13   
..         ...          ...     ...      ...        ...        ...   ...   
435      26945        12963       0    10330 2001-11-26 2004-06-11  2.46   
436      26945         1728       1    10248 2004-01-27 2005-06-21  5.27   
437      26945         9329       0    10154 2001-12-17 2004-07-22  5.65   
438      26945         4197       0    10333 2003-10-16 2005-07-10  4.50   
439      26945         3643       0    11434 2010-03-24 2011-12-22  0.13   

     loan_type_cash  loan_type_credit  loan_type_home  loan_type_other  
0             

#Add new and delete unimportant features

In [5]:
df = df_encoded

# Create new feature: monthly_payment which shows amount paid per month by client and loan_period which shows the period of the loan in month
loan_period_in_month = (df['loan_end'] - df['loan_start']) // pd.Timedelta(days=30.4)
total_loan_amount = df['loan_amount'] * (1 + df['rate']/100)
df_loan_period = loan_period_in_month
df_monthly_payment = total_loan_amount / loan_period_in_month

# Create new feature: interest_amount which shows the amount of interest for the loan
df_interest_amount = df['loan_amount'] * df['rate']/100

# Add new features to DataFrame
df['loan_period'] = df_loan_period
df['monthly_payment'] = df_monthly_payment
df['interest_amount'] = df_interest_amount

# Delete loan_Start, Loan_end, loan_id, client_id column
df = df_encoded.drop(columns=['loan_start', 'loan_end', 'loan_id', 'client_id'])

# Filter out the outliers
df = delete_outliers(df, 'loan_period')
df = delete_outliers(df, 'monthly_payment')
df = delete_outliers(df, 'interest_amount')
df = df.reset_index(drop=True)
# Check the result
print(df)
print(df.info())

     loan_amount  repaid  rate  loan_type_cash  loan_type_credit  \
0          13672       0  2.15             0.0               0.0   
1           9794       0  1.25             0.0               1.0   
2          12734       1  0.68             0.0               0.0   
3          12518       1  1.24             1.0               0.0   
4          14049       1  3.13             0.0               1.0   
..           ...     ...   ...             ...               ...   
422        12963       0  2.46             0.0               0.0   
423         1728       1  5.27             0.0               1.0   
424         9329       0  5.65             0.0               0.0   
425         4197       0  4.50             0.0               0.0   
426         3643       0  0.13             0.0               0.0   

     loan_type_home  loan_type_other  loan_period  monthly_payment  \
0               1.0              0.0           20       698.297400   
1               0.0              0.0       

#Data scaling

In [6]:
from sklearn.preprocessing import StandardScaler

# Columns to scale
columns_to_scale = ['loan_amount', 'rate', 'monthly_payment', 'interest_amount', 'loan_period']

# Initialize the Standard scaler
scaler = StandardScaler()

# Fit the scaler to the selected columns and transform the data
scaled_data = scaler.fit_transform(df[columns_to_scale])

# Convert the scaled data back to a DataFrame
scaled_df = pd.DataFrame(scaled_data, columns=columns_to_scale)

# Combine the scaled columns with the remaining columns in the original DataFrame
df_scaled = pd.concat([df.drop(columns=columns_to_scale), scaled_df], axis=1)

# Print the result
print(df_scaled)
print(df_scaled.info())

     repaid  loan_type_cash  loan_type_credit  loan_type_home  \
0         0             0.0               0.0             1.0   
1         0             0.0               1.0             0.0   
2         1             0.0               0.0             1.0   
3         1             1.0               0.0             0.0   
4         1             0.0               1.0             0.0   
..      ...             ...               ...             ...   
422       0             0.0               0.0             0.0   
423       1             0.0               1.0             0.0   
424       0             0.0               0.0             0.0   
425       0             0.0               0.0             1.0   
426       0             0.0               0.0             1.0   

     loan_type_other  loan_amount      rate  monthly_payment  interest_amount  \
0                0.0     1.405243 -0.399900         1.771941         0.322007   
1                0.0     0.468870 -0.812656         0.750

#Train and test split

In [7]:
from sklearn.model_selection import train_test_split

df = df_scaled

# Split the data into features (X) and target variable (y)
X = df.drop(columns=['loan_amount'])
y = df['loan_amount']

# Split the data into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Print the shape of the training and test sets
print("Training set shape", X_train.shape, y_train.shape)
print("Test set shape", X_test.shape, y_test.shape)

Training set shape (341, 9) (341,)
Test set shape (86, 9) (86,)


#Single variable linear regression

In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Prepare data
X_tr = X_train[['monthly_payment']]
y_tr = y_train
X_te = X_test[['monthly_payment']]

# Create and train the model using the training set
model = LinearRegression()
model.fit(X_tr, y_tr)

# Make predections on the test set
y_pred = model.predict(X_te)

# Calculate mean squared error and R-squared score on the test set
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("mse:", mse)
print("R-square:", r2)

mse: 0.1659746819620915
R-square: 0.8408791988858253


#Multivariable linear regression

In [9]:
# Prepare data
X_tr = X_train[['rate', 'monthly_payment', 'loan_period', 'interest_amount']]
y_tr = y_train
X_te = X_test[['rate', 'monthly_payment', 'loan_period', 'interest_amount']]

# Train the model using training set
model.fit(X_tr, y_tr)

# Make predections on the test set
y_pred = model.predict(X_te)

# Calculate mean squared error and R-squared score on the test set
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("mse:", mse)
print("R-square:", r2)


mse: 0.04229376438139068
R-square: 0.959452745497424


#Polynomial regression

In [10]:
from sklearn.preprocessing import PolynomialFeatures

# Prepare the data
X_tr = X_train[['monthly_payment']]
y_tr = y_train
X_te = X_test[['monthly_payment']]

# Create polynomial features up to degree 3 for training set
poly_features = PolynomialFeatures(degree=3)
X_tr_poly = poly_features.fit_transform(X_tr)

# Train the model
model.fit(X_tr_poly, y_tr)

# Transform test data into polynomial features
X_te_poly = poly_features.transform(X_te)

# Make predictions on the test set
y_pred = model.predict(X_te_poly)

# Calculate mean squared error and R-squared score on the test set
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("mse:", mse)
print("R-square:", r2)


mse: 0.10391140026607501
R-square: 0.9003795936839935
